# Вебинар 5. Домашнее задание

Само домашнее задание нахожится в конце ноутбука

---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.metrics import precision_at_k, recall_at_k
# from src.utils import prefilter_items

In [2]:
data = pd.read_csv('../data/retail_train.csv')



-----

# Домашнее задание

1. Изучите структуру модуля src
2. Перенесите функции prefilter_items и postfilter_items в модуль src.utils.py
3. Реализуйте функции get_similar_items_recommendation, get_similar_users_recommendation (они разбирались на вебинаре) и переместите в src.utils.py.  
postfilter можно пока оставить пустой (или как-то бороться с товаром 999999) - на Ваше усмотрение.  
В prefilter нужно реализовать закомментированные ограничения.
4. Создайте модуль src.recommenders.py. Напишите код для класса ниже и положите его в src.recommenders.py  
get_similar_items_recommendation и get_similar_users_recommendation - реализуйте их в качетсве методов класса.  
Посчитайте с помощью них рекоменадции и найдите precision@5
5. Проверьте, что все модули корректно импортируются
6. Если вы еще не прочитали [статью](https://habr.com/ru/company/hh/blog/347276/) о рекомендательных системах и поиске в hh.ru, то обязательно прочитайте

**Внимание!** модуль для вашего удобства - дополняйте его теми функциями, которые Вам пригодятся.  
Формальное задание только на prefiletr_items 

In [3]:
def prefilter_items_top_n_999999(data_train,n=5000):
    """Оставим только n самых популярных товаров, остальные переименуем в 999999"""
    df = data_train.copy()
    popularity = df.groupby('item_id')['quantity'].count().reset_index()
    popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
    top_n = popularity.sort_values('n_sold', ascending=False).head(n).item_id.tolist()
    df.loc[~df['item_id'].isin(top_n), 'item_id'] = 999999 
    return df
    
    
def prefilter_items_top_n_del(data_train,n=5000):
    """Оставим только n самых популярных товаров, транзакции с остальными товрами удалим"""
    df = data_train.copy()
    popularity = df.groupby('item_id')['quantity'].count().reset_index()
    popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
    top_n = popularity.sort_values('n_sold', ascending=False).head(n).item_id.tolist()
    df = df.loc[df['item_id'].isin(top_n)]  
    return df
    
    
def prefilter_items_not_top_n_del(data_train,n=5000):
    """транзакции с самыми не популярными n товрами удалим"""
    df = data_train.copy()
    not_popularity = df.groupby('item_id')['quantity'].count().reset_index()
    not_popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
    not_top_n = not_popularity.sort_values('n_sold').head(n).item_id.tolist()
    df = df.loc[~df['item_id'].isin(not_top_n)]  
    return df    
 

def prefilter_items_old_weeks_n_del(data_train,weeks = 52):
    """Удалим транзакции с товарами, которые не покупали более n недель"""
    df = data_train.copy()
    old_item = df.groupby('item_id')['week_no'].max().reset_index()
    old_item = old_item.loc[old_item['week_no']>weeks,'item_id'].tolist()
    df = df.loc[df['item_id'].isin(old_item)]  
    return df


    
    # Уберем товары, которые не продавались за последние 12 месяцев
    
    # Уберем не интересные для рекоммендаций категории (department)
    
    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб. 
    
    # Уберем слишком дорогие товары
    


def postfilter_items():
    pass

In [4]:
def train_test_split(data,test_size_num,split_column):
    data_train = data[data[split_column] < data[split_column].max() - test_size_num]
    data_test = data[data[split_column] >= data[split_column].max() - test_size_num]
    return data_train, data_test

In [5]:
def get_similar_items_recommendation(df,users,model,not_my=0, N=5):

    """Рекомендуем товары, похожие на топ-N купленных юзером товаров
            not_my =1 если хотим предсказать поекупку собственных товаров (вроде own_recomender), 0 - обратно
    """

    assert  type(users)==list,'параметр users должен быть list'
    assert  not_my in [0,1],'параметр not_my должен быть равен 0 или 1'

    def get_recs(model,user,popularity,not_my=0):
        result = []
        for item in popularity[popularity['user_id']==user]['item_id'].to_list():
            recs_ = model.similar_items(itemid_to_id[item], N=3)
            recs = [id_to_itemid[i[0]] for i in recs_]
            if 999999 in recs:
                recs.remove(999999)
            result.append(recs[not_my])
        return  result



    my_data = df.copy()     
    my_data = my_data[my_data['user_id'].isin(users)]    
    popularity = my_data.groupby(['user_id', 'item_id'])['quantity'].count().reset_index()
    popularity.sort_values('quantity', ascending=False, inplace=True)
    popularity = popularity[popularity['item_id'] != 999999]
    popularity =popularity.groupby('user_id').head(N)
    popularity.sort_values(['user_id','quantity'], ascending=False, inplace=True)
    result = pd.DataFrame()
    result['user_id'] = users
    result['similar_recommendation'] = result['user_id'].apply(\
                            lambda x: get_recs(model=model,user=x,popularity = popularity,not_my=not_my))

    return result

In [6]:
   
def get_similar_users_recommendation(model,uim,users, params={'filter_already_liked_items':False, 
                        'filter_items':[999999], 
                        "recalculate_user":True}, N=5):
    """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
    
    assert  type(users)==list,'параметр users должен быть list'
    
    def get_user(user):
        users = model.similar_users(userid_to_id[user], N=2)
        
        return  id_to_userid[users[1][0]]
    
    
    def predict_als(users,N=5):
        
        param = params.copy()
        assert type(users) == list, 'users - должен быть списком'
        param['user_items'] = uim
        param['N'] = N
        answer = pd.DataFrame()
        answer['user_id']=users
        if param['filter_items']:
            param['filter_items']=[itemid_to_id[i] for i in params['filter_items']]
        rec=[]
        for user in users:
            param['userid'] = userid_to_id[user]
            rec.append( [id_to_itemid[i[0]] for i in model.recommend(**param)])
        answer['result']  = rec
        return answer

    
    result = pd.DataFrame()
    result['user_id'] = users
    result['simular_user_id'] = result['user_id'].apply(get_user)
    result['similar_recommendation'] = predict_als(result['simular_user_id'].to_list(),N=5)['result']

    return result

In [7]:
data.tail(3)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
2396801,1001,41655829421,663,13217800,1,1.69,3131,0.00,2231,95,0.0,0.0
2396802,1167,41656790510,663,6410462,22451,43.98,3385,-0.65,1059,95,0.0,0.0
2396803,1167,41656790510,663,6430664,1,8.00,3385,0.00,1059,95,0.0,0.0


In [8]:
%%time
data_train,data_test = train_test_split(data,3,'week_no')
test_size_weeks = 3
data_train=prefilter_items_top_n_999999(data_train)
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float)
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))
user_item_matrix = bm25_weight(user_item_matrix.T).T
model = AlternatingLeastSquares(factors=100, 
                                regularization=35,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=-1)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']



Wall time: 7.75 s


In [9]:
%%time
get_similar_items_recommendation(data_train,[1,2,3,4,5,6,7,8,9,10],model,not_my=1)

Wall time: 110 ms


,user_id,similar_recommendation
0,1,"[957164, 981760, 1029743, 1007512, 9526411]"
1,2,"[1106523, 1133018, 8090537, 5569845, 985999]"
2,3,"[1053690, 1092026, 883932, 1075979, 951590]"
3,4,"[951590, 847066, 1048462, 1111786, 1075368]"
4,5,"[1058997, 1070702, 5995503, 5569327, 1103618]"
5,6,"[981760, 825541, 1082185, 845208, 948650]"
6,7,"[981760, 1019247, 1015247, 1050229, 5583595]"
7,8,"[981760, 1106523, 1037863, 1110843, 5569845]"
8,9,"[995242, 882489, 1098066, 826249, 862799]"
9,10,"[1133018, 1012873, 1012873, 1138443, 930917]"


In [10]:
%%time
users = result['user_id'].to_list()
result['item_similar_recommendation'] = get_similar_items_recommendation(data_train,users,model,not_my=1)['similar_recommendation']
result.apply(lambda row: precision_at_k(row['item_similar_recommendation'], row['actual']), axis=1).mean()


Wall time: 3.24 s


0.14084231145935228

In [11]:
uim = csr_matrix(user_item_matrix).tocsr()
get_similar_users_recommendation(model,uim,[1,2,3,4,5,6,7,8,9,10],  N=5)

,user_id,simular_user_id,similar_recommendation
0,1,2171,"[861272, 1005186, 970619, 976214, 1115576]"
1,2,475,"[855356, 1092040, 1053690, 968146, 1036936]"
2,3,490,"[944466, 872417, 5568378, 1074754, 822407]"
3,4,662,"[1115098, 6773204, 1070272, 1003836, 1045220]"
4,5,2495,"[5995609, 1067419, 9420336, 9573206, 867931]"
5,6,521,"[848071, 1028816, 819840, 1025650, 962274]"
6,7,1512,"[1404121, 9802540, 1136988, 912553, 900698]"
7,8,2468,"[860703, 1082185, 1021116, 1130944, 1121367]"
8,9,1952,"[887962, 1066893, 995502, 1085256, 8359154]"
9,10,2101,"[902192, 968057, 872702, 1041476, 9526313]"


In [12]:
%%time
users = result['user_id'].to_list()
result['users_similar_recommendation'] = get_similar_users_recommendation(model,uim,users)['similar_recommendation']
result.apply(lambda row: precision_at_k(row['users_similar_recommendation'], row['actual']), axis=1).mean()

Wall time: 26 s


0.08393731635651303

In [13]:
data.user_id.nunique(),data.item_id.nunique()

(2499, 89051)

In [14]:
a=prefilter_items_top_n_999999(data)
a.user_id.nunique(),a.item_id.nunique()

(2499, 5001)

In [15]:
a=prefilter_items_top_n_del(data)
a.user_id.nunique(),a.item_id.nunique()

(2483, 5000)

In [16]:
a=prefilter_items_not_top_n_del(data,n=70000)
a.user_id.nunique(),a.item_id.nunique()

(2498, 19051)

In [17]:
a=prefilter_items_old_weeks_n_del(data)
a.user_id.nunique(),a.item_id.nunique()

(2498, 61908)

In [18]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class MainRecommender:
    
    
    
    
    own_recomender_defult_param = {'filter_already_liked_items':False, 
                        'filter_items':False, 
                        "recalculate_user":True}
    
    model_als_defult_param ={'factors':50, 'regularization':15, 'iterations':15, 
                             'num_threads':-1,'calculate_training_loss':False}
    
    def __init__(self, data,data_test=None,split_info=None):
        """ data - dataframe c данными
            data_test - даные для валидации, если нет и есть split_info то создаем
            split_info кортеж с инфрмацией как создать data_test (размер, поле деления) рассматривается только в слуяае отсутвя 
            data_test
        """     
        
        self.data_validation={}
        self.data_validation['status'] = False
        self.user_item_matrix = {'status':False,'matrix':None,'params':None}
        self.own_recommender_is_fit= {'status':False,'params':None}
        self.als_recommender_is_fit= {'status':False,'params':None}
        self.data = data.copy()
        self.full_data_train = data.copy() #Оставим полный объем данный , если нужно будет предсказывать по полному объему данных
        self.data_train = data.copy()
        if data_test is not None:
            self.data_test = data_test.copy()
        else:
            self.data_test = None
            if split_info:
                self.data_train,self.data_test = self.train_test_split(test_size_num = split_info[0],split_column =split_info[1])
        if  self.data_test is not None:
            self.data_validation['data'] = self.get_validation_data()
            self.data_validation['status'] = True


 
    def prefiltr_1(self,my_data,n=5000):
        df = my_data.copy()
        """Оставим только 5000 самых популярных товаров остальные переименуем в 999999"""
        popularity = my_data.groupby('item_id')['quantity'].count().reset_index()
        popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
        top_5000 = popularity.sort_values('n_sold', ascending=False).head(n).item_id.tolist()
        df.loc[~df['item_id'].isin(top_5000), 'item_id'] = 999999 
        return df
    
    
    def prefiltr_2(self,data_train,n=5000):
        """Оставим только n самых популярных товаров, транзакции с остальными товрами удалим"""
        df = data_train.copy()
        popularity = df.groupby('item_id')['quantity'].count().reset_index()
        popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
        top_n = popularity.sort_values('n_sold', ascending=False).head(n).item_id.tolist()
        df = df.loc[df['item_id'].isin(top_n)]  
        return df
    
    
    def prefiltr_3(self,data_train,n=5000):
        """транзакции с самыми не популярными n товрами удалим"""
        df = data_train.copy()
        not_popularity = df.groupby('item_id')['quantity'].count().reset_index()
        not_popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
        not_top_n = not_popularity.sort_values('n_sold').head(n).item_id.tolist()
        df = df.loc[~df['item_id'].isin(not_top_n)]  
        return df   
    
    
    def prefiltr_4(self,data_train,weeks = 52):
        """Удалим транзакции с товарами, которые не покупали более n недель"""
        df = data_train.copy()
        old_item = df.groupby('item_id')['week_no'].max().reset_index()
        old_item = old_item.loc[old_item['week_no']>weeks,'item_id'].tolist()
        df = df.loc[df['item_id'].isin(old_item)]  
        return df
    


  
    def train_test_split(self,test_size_num,split_column):
        data_train = self.data[self.data[split_column] < self.data[split_column].max() - test_size_num]
        data_test = self.data[self.data[split_column] >= self.data[split_column].max() - test_size_num]
        return data_train, data_test
    
    
   
    def get_validation_data(self):
        result = self.data_test.groupby('user_id')['item_id'].unique().reset_index()
        result['train'] = result['user_id'].map(self.data_train.groupby('user_id')['item_id'].unique())
        result['full_train'] = result['user_id'].map(self.full_data_train.groupby('user_id')['item_id'].unique())
        result.rename(columns={'item_id':'test'},inplace=True)
        return result

 
    def prepare_matrix(self,agg_column,full=None,filtr=None):
        my_data = self.data_train.copy()
        if full:
            my_data = self.full_data_train.copy()
        if  filtr:
            for i in filtr:
                prefiltr = 'self.prefiltr_'+str(i)+'(my_data)'
                my_data = eval(prefiltr)
            
        user_item_matrix = pd.pivot_table(my_data, 
                              index='user_id', columns='item_id', 
                              values=agg_column[0], 
                              aggfunc=agg_column[1], 
                              fill_value=0
                             )
        
        user_item_matrix = user_item_matrix.astype(float) 
        self.prepare_dicts(user_item_matrix)
        self.current_working_data = my_data.copy()

        return user_item_matrix
        

        return user_item_matrix
            


    def prepare_dicts(self,user_item_matrix):
        """Подготавливает вспомогательные словари"""
        
        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        self.id_to_itemid = dict(zip(matrix_itemids, itemids))
        self.id_to_userid = dict(zip(matrix_userids, userids))

        self.itemid_to_id = dict(zip(itemids, matrix_itemids))
        self.userid_to_id = dict(zip(userids, matrix_userids))
        
        return  self.id_to_itemid,  self.id_to_userid,  self.itemid_to_id,  self.userid_to_id
    
    
     
    def make_data(self,agg_column,filtr=None,full =False):
        self.full = full
        uim = self.prepare_matrix(agg_column=agg_column,full=full,filtr=filtr)
        uim_w = uim.copy()
        self.user_item_matrix['uim_matrix_w'] = csr_matrix(uim_w).tocsr()
        uim[uim>0]=1
        self.user_item_matrix['uim_matrix'] = csr_matrix(uim).tocsr()
        
        self.user_item_matrix['ium_matrix_w_tfidf'] = tfidf_weight(csr_matrix(uim_w.T).tocsr())
        self.user_item_matrix['ium_matrix_tfidf'] = tfidf_weight(csr_matrix(uim.T).tocsr())
        self.user_item_matrix['ium_matrix_w_bm25'] = bm25_weight(csr_matrix(uim_w.T).tocsr())
        self.user_item_matrix['ium_matrix_bm25'] = bm25_weight(csr_matrix(uim.T).tocsr())

        self.user_item_matrix['status'] = True
        self.user_item_matrix['params'] = {'agg_column':agg_column,'filtr':filtr,'full':full}
        return self.user_item_matrix
            
        
    def precision_at_k(x, k=5):
    
        bought_list = np.array(x['test'])
        recommended_list = np.array(x['predict'])[:k]
        if len(recommended_list) == 0:
            return 0


        flags = np.isin(bought_list, recommended_list)
        precision = flags.sum() / len(recommended_list)


        return precision
        
        
    
    def fit_own_recommender(self,weighting=False):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""
        
        assert self.user_item_matrix['status'], 'необходимо сначала выполнить метод make_data(self,agg_column,filtr=None,weighting=None,full =False)'
        ium = self.user_item_matrix['uim_matrix'].T
        if weighting:
            assert (weighting == 'tf_idf' or weighting == 'bm25'), 'необходимо указать weighting: tf_idf или bm25 или None'
            if  weighting == 'tf_idf':
                ium = self.user_item_matrix['ium_matrix_tfidf']
            else:
                ium = self.user_item_matrix['ium_matrix_bm25']   
        self.own_recommender = ItemItemRecommender(K=1, num_threads=-1)
        self.own_recommender.fit(ium)      
        self.own_recommender_is_fit['status'] =True
        self.own_recommender_is_fit['params'] ={'model':'ItemItemRecommender(K=1, num_threads=-1)','weighting':weighting}
        self.own_recommender_is_fit['ium']=ium
        
        return self.own_recommender
    
    
    def predict_own_recommender(self,users,N=5,params=own_recomender_defult_param):
        
        param = params.copy()
        assert self.own_recommender_is_fit['status'], 'необходимо сначала выполнить метод fit_own_recommender()'
        assert type(users) == list, 'users - должен быть списком'
        uim = self.user_item_matrix['uim_matrix']
        param['user_items'] = uim
        param['N'] = N
        answer = pd.DataFrame()
        answer['user_id']=users
        if param['filter_items']:
            param['filter_items']=[self.itemid_to_id[i] for i in params['filter_items']]
        rec=[]
        for user in users:
            param['userid'] = self.userid_to_id[user]
            rec.append( [self.id_to_itemid[i[0]] for i in self.own_recommender.recommend(**param)])
        answer['result']  = rec
        return answer

    
    
    def validation_own_recommender(self,metric=precision_at_k,N=5,params=own_recomender_defult_param):
        assert self.data_validation['status'], 'тестовые данные не созданы'
        assert self.own_recommender_is_fit['status'], 'необходимо сначала выполнить метод fit_own_recommender()'
        df = self.data_validation['data']
        users = df['user_id'].to_list()
        predict = self.predict_own_recommender(users = users,N=N,params=params)
        df['predict'] = predict['result']
        
        return df.apply(metric,axis=1).mean()
            
        
  
    def fit_als(self, params = model_als_defult_param,weighting=False):
        """Обучает ALS"""
        
        assert self.user_item_matrix['status'], 'необходимо сначала выполнить метод make_data(self,agg_column,filtr=None,weighting=None,full =False)'
        ium = self.user_item_matrix['uim_matrix_w'].T
        if weighting:
            assert (weighting == 'tf_idf' or weighting == 'bm25'), 'необходимо указать weighting: tf_idf или bm25 или None'
            if  weighting == 'tf_idf':
                ium = self.user_item_matrix['ium_matrix_w_tfidf']
            else:
                ium = self.user_item_matrix['ium_matrix_w_bm25']
        
        self.model_als = AlternatingLeastSquares(**params)
        self.model_als.fit(ium)
        self.als_recommender_is_fit['status'] = True
        self.als_recommender_is_fit['params'] = {'model':params,'weighting':weighting}
        self.als_recommender_is_fit['ium'] = ium
        
        return self.model_als
    
    
    def predict_als(self,users,N=5,params=own_recomender_defult_param):
        
        param = params.copy()
        assert self.als_recommender_is_fit['status'], 'необходимо сначала выполнить метод fit_als()'
        assert type(users) == list, 'users - должен быть списком'
        uim = self.user_item_matrix['uim_matrix_w']
        param['user_items'] = uim
        param['N'] = N
        answer = pd.DataFrame()
        answer['user_id']=users
        if param['filter_items']:
            param['filter_items']=[self.itemid_to_id[i] for i in params['filter_items']]
        rec=[]
        for user in users:
            param['userid'] = self.userid_to_id[user]
            rec.append( [self.id_to_itemid[i[0]] for i in self.model_als.recommend(**param)])
        answer['result']  = rec
        return answer
    
    
    def validation_als_recommender(self,metric=precision_at_k,N=5,params=own_recomender_defult_param):
        assert self.data_validation['status'], 'тестовые данные не созданы'
        assert self.als_recommender_is_fit['status'], 'необходимо сначала выполнить метод fit_als()'
        df = self.data_validation['data'].copy()
        users = df['user_id'].to_list()
        predict = self.predict_als(users = users,N=N,params=params)
        df['predict'] = predict['result']

        return df.apply(metric,axis=1).mean()  
    
    
    def get_recs(self,user,popularity,not_my=0):
        result = []
        for item in popularity[popularity['user_id']==user]['item_id'].to_list():
            recs_ = self.model_als.similar_items(self.itemid_to_id[item], N=3)
            recs = [self.id_to_itemid[i[0]] for i in recs_]
            if 999999 in recs:
                recs.remove(999999)
            result.append(recs[not_my])
        return  result      


    def get_similar_items_recommendation(self, users,not_my=0, N=5):
        
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров
        not_my =1 если хотим предсказать поекупку собственных товаров (вроде own_recomender), 0 - обратно"""
        assert  self.als_recommender_is_fit['status'],'Модель als не обучена, используйте fit_als()'
        assert  type(users)==list,'параметр users должен быть list'
        assert  not_my in [0,1],'параметр not_my должен быть равен 0 или 1'
        my_data = self.current_working_data.copy()
        my_data = my_data[my_data['user_id'].isin(users)]    
        popularity = my_data.groupby(['user_id', 'item_id'])['quantity'].count().reset_index()
        popularity.sort_values('quantity', ascending=False, inplace=True)
        popularity = popularity[popularity['item_id'] != 999999]
        popularity =popularity.groupby('user_id').head(N)
        popularity.sort_values(['user_id','quantity'], ascending=False, inplace=True)
        result = pd.DataFrame()
        result['user_id'] = users
        result['similar_recommendation'] = result['user_id'].apply(\
                                            lambda x: self.get_recs(user = x,popularity = popularity,not_my=not_my))

        return result
    
    
    def validation_similar_items_recommendation(self,metric=precision_at_k,N=5,not_my=0):
        assert self.data_validation['status'], 'тестовые данные не созданы'
        assert self.als_recommender_is_fit['status'], 'необходимо сначала выполнить метод fit_als()'
        assert  not_my in [0,1],'параметр not_my должен быть равен 0 или 1'
        df = self.data_validation['data'].copy()
        users = df['user_id'].to_list()
        predict = self.get_similar_items_recommendation(users = users,N=N,not_my=not_my)
        df['predict'] = predict['similar_recommendation']

        return df.apply(metric,axis=1).mean() 
    
    
    
    def get_user(self,user):
        users = self.model_als.similar_users(self.userid_to_id[user], N=2)
        
        return  self.id_to_userid[users[1][0]]
    
    
    def get_similar_users_recommendation(self, users, N=5,params=own_recomender_defult_param):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
        assert  self.als_recommender_is_fit['status'],'Модель als не обучена, используйте fit_als()'
        assert  type(users)==list,'параметр users должен быть list'
        result = pd.DataFrame()
        result['user_id'] = users
        result['simular_user_id'] = result['user_id'].apply(self.get_user)
        result['similar_recommendation'] = self.predict_als(result['simular_user_id'].to_list(),N=5,params=params)['result']

        return result    
            
    def validation_similar_users_recommendation(self,metric=precision_at_k,N=5):
        assert self.data_validation['status'], 'тестовые данные не созданы'
        assert self.als_recommender_is_fit['status'], 'необходимо сначала выполнить метод fit_als()'
        df = self.data_validation['data'].copy()
        users = df['user_id'].to_list()
        predict = self.get_similar_users_recommendation(users = users,N=N)
        df['predict'] = predict['similar_recommendation']

        return df.apply(metric,axis=1).mean()     
    


Работа класса

In [19]:
%%time
# создаем объект класса. data  - полный набор данных,  
# split_info - кортеж, как разбить данные на трайн и тест, если нужно. 
# Если есть тестовые данные их можно передать в парамаетр data_test 
# week_no - поле по которому делим трейн и тест, 3 значение отсечения

my_recomender = MainRecommender(data=data,split_info=(3,'week_no'))

Wall time: 1.01 s


In [20]:
%%time
# создадим все маттрицы для работы
# agg_column - поле агрегации и тип агрегации для сводной таблицы
# filtr - указываем каким префильтрам подвергнуть матрицу при создании.Это список. Пока имеется только префильтр _1 отбор 5000 
# самых популярных товаров. Замена остальных на 999999
# full - если нужно сдлеать матицу из полного набьора данных, если Fals то делает из тренировочного,
# полученого делением полного на трайн и тест
my_recomender.make_data(agg_column=('quantity','count'),filtr=[1],full =False)

Wall time: 3.44 s


{'status': True,
 'matrix': None,
 'params': {'agg_column': ('quantity', 'count'), 'filtr': [1], 'full': False},
 'uim_matrix_w': <2499x5001 sparse matrix of type '<class 'numpy.float64'>'
 	with 679365 stored elements in Compressed Sparse Row format>,
 'uim_matrix': <2499x5001 sparse matrix of type '<class 'numpy.float64'>'
 	with 679365 stored elements in Compressed Sparse Row format>,
 'ium_matrix_w_tfidf': <5001x2499 sparse matrix of type '<class 'numpy.float64'>'
 	with 679365 stored elements in COOrdinate format>,
 'ium_matrix_tfidf': <5001x2499 sparse matrix of type '<class 'numpy.float64'>'
 	with 679365 stored elements in COOrdinate format>,
 'ium_matrix_w_bm25': <5001x2499 sparse matrix of type '<class 'numpy.float64'>'
 	with 679365 stored elements in COOrdinate format>,
 'ium_matrix_bm25': <5001x2499 sparse matrix of type '<class 'numpy.float64'>'
 	with 679365 stored elements in COOrdinate format>}

In [21]:
# Посмотрим что теперь имеется в классе
my_recomender.data # - полный наьор данных
my_recomender.data_test #- тестовый набор данных
my_recomender.data_train #- набор данных для обучения
my_recomender.userid_to_id
my_recomender.itemid_to_id
my_recomender.id_to_itemid
my_recomender.id_to_userid
None


In [22]:
# Набор данных для валидации на тесте , трейне
my_recomender.data_validation['data']

,user_id,test,train,full_train
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[825123, 831447, 840361, 845307, 852014, 85498...","[825123, 831447, 840361, 845307, 852014, 85498..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[866211, 878996, 882830, 904360, 921345, 93194...","[866211, 878996, 882830, 904360, 921345, 93194..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[840361, 851494, 851819, 851903, 863447, 87623...","[840361, 851494, 851819, 851903, 863447, 87623..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[865569, 886703, 889731, 893400, 995436, 10205...","[865569, 886703, 889731, 893400, 995436, 10205..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[824550, 824555, 826578, 835576, 841220, 84798...","[824550, 824555, 826578, 835576, 841220, 84798..."
...,...,...,...,...
2037,2496,[6534178],"[840361, 852159, 871756, 886703, 899624, 91612...","[840361, 852159, 871756, 886703, 899624, 91612..."
2038,2497,"[1016709, 9835695, 1132298, 16809501, 845294, ...","[838220, 1037840, 1052294, 5569230, 8090537, 1...","[838220, 1037840, 1052294, 5569230, 8090537, 1..."
2039,2498,"[15716530, 834484, 901776, 914190, 958382, 972...","[824555, 835576, 901776, 904023, 911215, 91749...","[824555, 835576, 901776, 904023, 911215, 91749..."
2040,2499,"[867188, 877580, 902396, 914190, 951590, 95813...","[838186, 853197, 864143, 883665, 932949, 93383...","[838186, 853197, 864143, 883665, 932949, 93383..."


In [23]:
# 4 матрицы в наборе а так же указаны параметры агрегации и фильтрации
my_recomender.user_item_matrix

{'status': True,
 'matrix': None,
 'params': {'agg_column': ('quantity', 'count'), 'filtr': [1], 'full': False},
 'uim_matrix_w': <2499x5001 sparse matrix of type '<class 'numpy.float64'>'
 	with 679365 stored elements in Compressed Sparse Row format>,
 'uim_matrix': <2499x5001 sparse matrix of type '<class 'numpy.float64'>'
 	with 679365 stored elements in Compressed Sparse Row format>,
 'ium_matrix_w_tfidf': <5001x2499 sparse matrix of type '<class 'numpy.float64'>'
 	with 679365 stored elements in COOrdinate format>,
 'ium_matrix_tfidf': <5001x2499 sparse matrix of type '<class 'numpy.float64'>'
 	with 679365 stored elements in COOrdinate format>,
 'ium_matrix_w_bm25': <5001x2499 sparse matrix of type '<class 'numpy.float64'>'
 	with 679365 stored elements in COOrdinate format>,
 'ium_matrix_bm25': <5001x2499 sparse matrix of type '<class 'numpy.float64'>'
 	with 679365 stored elements in COOrdinate format>}

### fit_own_recommender

In [24]:
# Обучим own_recommender без взешивания
my_recomender.fit_own_recommender()

In [25]:
# параметры предсказаний
params = {'filter_already_liked_items':False, 
                        'filter_items':[999999], 
                        "recalculate_user":True}

In [26]:
# сделаем 5 предсказания для юзеров 1 и 2
my_recomender.predict_own_recommender([1,2],5,params=params)

,user_id,result
0,1,"[1082185, 1029743, 1004906, 5978656, 1081177]"
1,2,"[1082185, 1098066, 1127831, 1004906, 1081177]"


In [27]:
# посмотрим  валидацию на тесте
my_recomender.validation_own_recommender(params=params)

0.19711067580802874

In [28]:
# Обучим own_recommender tf_idf
my_recomender.fit_own_recommender(weighting ='tf_idf')

In [29]:
# посмотрим  валидацию на тесте
my_recomender.validation_own_recommender(params=params)

0.21101860920665738

In [30]:
# Обучим own_recommender bm25
my_recomender.fit_own_recommender(weighting ='bm25')

In [31]:
%%time
# посмотрим  валидацию на тесте
my_recomender.validation_own_recommender(params=params)

Wall time: 146 ms


0.06520568070519137

### als recomender

In [32]:
# Обучим модель als
param_als = {'factors':1100, 'regularization':35, 'iterations':20, 
                             'num_threads':-1,'calculate_training_loss':True}


In [33]:
%%time
my_recomender.fit_als(params=param_als,weighting ='tf_idf')


Wall time: 15.5 s


In [34]:
%%time
# предикт als
my_recomender.predict_als(users = [1,2,3,4,5,6,7,8,9,10],N=5,params=params)

Wall time: 10.1 s


,user_id,result
0,1,"[995242, 1082185, 9527290, 840361, 940947]"
1,2,"[1133018, 1106523, 5569230, 1082185, 1098066]"
2,3,"[1106523, 951590, 1133018, 910032, 1092026]"
3,4,"[883932, 962229, 883404, 1075368, 5569230]"
4,5,"[1082185, 1126899, 1029743, 916122, 5568378]"
5,6,"[1082185, 878996, 1024306, 834484, 840361]"
6,7,"[1082185, 1106523, 1122358, 1127831, 1133018]"
7,8,"[1082185, 826249, 1133018, 1029743, 1004906]"
8,9,"[1029743, 1082185, 995242, 1098066, 826249]"
9,10,"[1106523, 1082185, 1133018, 1113588, 1005186]"


In [35]:
%%time
# oценка валидации als на тесте precision@5 tf_idf
my_recomender.validation_als_recommender(params=params)

Wall time: 38min 31s


0.30293829578844245

In [36]:
# обучим als без взвешивания
my_recomender.fit_als(params=param_als)

In [37]:
%%time
# oценка валидации als на тесте precision@5 без взвешивания
my_recomender.validation_als_recommender(params=params)

Wall time: 31min 44s


0.2791380999020556

In [38]:
# обучим als bm25
my_recomender.fit_als(params=param_als,weighting ='bm25')

In [39]:
%%time
# oценка валидации als на тесте precision@5  bm52
my_recomender.validation_als_recommender(params=params)

Wall time: 32min 34s


0.34427032321253753

### simular item and simular user

In [40]:
%%time
# item_item рекомендации not_my=0 (собственные) bm52
my_recomender.get_similar_items_recommendation( users = [1,2,3,4,5,6,7,8,9,10], N=5,not_my=0)


Wall time: 91.1 ms


,user_id,similar_recommendation
0,1,"[856942, 1082185, 995242, 940947, 9527290]"
1,2,"[1133018, 1106523, 8090521, 5569230, 916122]"
2,3,"[1092026, 1053690, 951590, 998206, 910032]"
3,4,"[883932, 891423, 962229, 910109, 1052294]"
4,5,"[1126899, 874972, 5995609, 5568378, 1123022]"
5,6,"[1082185, 1119051, 840361, 1037863, 863447]"
6,7,"[1082185, 1122358, 6944571, 1022003, 828867]"
7,8,"[1082185, 1133018, 969932, 1005186, 5569230]"
8,9,"[1029743, 862799, 826249, 1098066, 1056005]"
9,10,"[1106523, 945779, 1113588, 883003, 892008]"


In [41]:
%%time
# item_item рекомендации валидация на тесте precision@5 'bm25' not_my=0 (собственные)
my_recomender.validation_similar_items_recommendation(not_my=0)

Wall time: 5.57 s


0.4002693437806084

In [42]:
%%time
# item_item рекомендации not_my=1 (схожие товары) собственные bm52
my_recomender.get_similar_items_recommendation( users = [1,2,3,4,5,6,7,8,9,10], N=5,not_my=1)

Wall time: 81.1 ms


,user_id,similar_recommendation
0,1,"[1045586, 981760, 1029743, 5582712, 15926844]"
1,2,"[1106523, 1133018, 8090537, 1021324, 985999]"
2,3,"[844165, 5569230, 13841744, 1075979, 1098066]"
3,4,"[951590, 992292, 1048462, 1111786, 1075368]"
4,5,"[1058997, 12262778, 5995503, 5569327, 1103618]"
5,6,"[981760, 825541, 1082185, 995785, 948650]"
6,7,"[981760, 1038985, 1015247, 1050229, 1110218]"
7,8,"[981760, 1106523, 845208, 1110843, 1021324]"
8,9,"[995242, 1056005, 1098066, 826249, 862799]"
9,10,"[1133018, 1012873, 918733, 1138443, 1045220]"


In [43]:
%%time
# item_item рекомендации валидация на тесте precision@5 'bm25' not_my=1 (схожие товары)
my_recomender.validation_similar_items_recommendation(not_my=1)

Wall time: 5.68 s


0.16131243878550247

In [44]:
%%time
# user_item рекомендации
my_recomender.get_similar_users_recommendation( users=[1,2,3,4,5,6,7,8,9,10], N=5,params=params)

Wall time: 5.58 s


,user_id,simular_user_id,similar_recommendation
0,1,1424,"[1082185, 995242, 1024306, 1029743, 840361]"
1,2,475,"[995242, 1029743, 1082185, 981760, 5569845]"
2,3,1883,"[1092026, 883404, 1053690, 8090521, 859075]"
3,4,662,"[883404, 1070272, 983584, 995242, 1029743]"
4,5,2495,"[1082185, 867931, 906838, 1126899, 1029968]"
5,6,1034,"[1082185, 994928, 1058997, 1126899, 849843]"
6,7,2163,"[1082185, 862349, 908531, 9338009, 1058997]"
7,8,586,"[1029743, 995242, 1106523, 1053690, 6534178]"
8,9,930,"[1016800, 917033, 1050741, 850102, 926905]"
9,10,2101,"[995242, 1082185, 1029743, 849843, 981760]"


In [45]:
%%time
# user_item валидация на тесте precision@5 'bm25'
my_recomender.validation_similar_users_recommendation()

Wall time: 25min 28s


0.13114593535749156

In [46]:
# обучим als без взвешивания
my_recomender.fit_als(params=param_als)

In [47]:
%%time
# item_item рекомендации валидация на тесте precision@5 без взвешивания not_my=0 (собственные)
my_recomender.validation_similar_items_recommendation(not_my=0)

Wall time: 6.01 s


0.4002693437806084

In [48]:
%%time
# item_item рекомендации валидация на тесте precision@5 без взвешивания not_my=1 (альтернативные)
my_recomender.validation_similar_items_recommendation(not_my=1)

Wall time: 5.85 s


0.15631733594515035

In [49]:
%%time
# user_item валидация на тесте precision@5 без взвешиания
my_recomender.validation_similar_users_recommendation()

Wall time: 25min 5s


0.18628795298726486

In [50]:
%%time
# обучим als tf_idf
my_recomender.fit_als(params=param_als,weighting ='tf_idf')


Wall time: 13.4 s


In [51]:
%%time
# item_item рекомендации валидация на тесте precision@5 без взвешивания not_my=0
my_recomender.validation_similar_items_recommendation(not_my=0)

Wall time: 5.57 s


0.4002693437806084

In [52]:
%%time
# item_item рекомендации валидация на тесте precision@5 без взвешиванияnot_my=1
my_recomender.validation_similar_items_recommendation(not_my=1)

Wall time: 5.52 s


0.14191968658178125

In [53]:
%%time
# user_item валидация на тесте precision@5 без взвешиания
my_recomender.validation_similar_users_recommendation()

Wall time: 31min 28s


0.1549461312438767

----